Data Processing

To convert all data into numerical form

In [ ]:
import json
for i in range(2015,2024):
    data1 = None
    data2 = None
    with open(f'../dataset/fixtures_epl/fixtures_epl_{i}.json') as f:
        data1 = json.load(f)
    with open(f'../dataset/epl_stats/epl_stats_{i}.json') as f:
        data2 = json.load(f)

    def find_fulltime_score_in_dict(dictionary, fixture_id):
        for fixture in dictionary:
            if fixture['fixture']['id'] == fixture_id:
                return {'home_fulltime_score': fixture['score']['fulltime']['home'],
                        'away_fulltime_score': fixture['score']['fulltime']['away']}


    final_data = []
    for fixture in data2:
        converted_fixture = {
            "fixture_id": fixture['fixture_id'],
            "timestamp": fixture["timestamp"],
            "home_team": fixture["home_team"],
            "home_team_id": fixture["home_team_id"],
            "away_team": fixture["away_team"],
            "away_team_id": fixture["away_team_id"],
        }
        fulltime_scores = find_fulltime_score_in_dict(data1, converted_fixture["fixture_id"])
        converted_fixture['home_fulltime_score'] = fulltime_scores['home_fulltime_score']
        converted_fixture['away_fulltime_score'] = fulltime_scores['away_fulltime_score']

        #for home stats
        home_stats = fixture["home_stats"]["statistics"]
        for stat in home_stats:
            #put home in front of the stat. change to lower case with underscore joining instead of spaces
            new_label = 'home_' + stat['type'].lower().replace(' ','_')

            if stat['type'] in ["Total passes", "Passes accurate","away_expected_goals","home_expected_goals"]:
                pass #no need to do
            elif stat['type'] == "Ball Possession":
                converted_fixture['home_ball_possession_%'] = int(stat['value'][:-1])
            elif stat['type'] == "Yellow Cards" or stat['type'] == "Red Cards" or stat['type'] == "Goalkeeper Saves":
                if stat['value'] != None:
                    converted_fixture[new_label] = stat['value']
                else:
                    converted_fixture[new_label] = 0
            elif stat['type'] == "Passes %":
                converted_fixture['home_passing_accuracy_%'] = int(stat['value'][:-1])
            else:
                if stat['value'] == None:
                    converted_fixture[new_label] = 0
                else:
                    converted_fixture[new_label] = stat['value']

        #for away stats
        away_stats = fixture["away_stats"]["statistics"]
        for stat in away_stats:
            #put home in front of the stat. change to lower case with underscore joining instead of spaces
            new_label = 'away_' + stat['type'].lower().replace(' ','_')

            if stat['type'] in ["Total passes", "Passes accurate","away_expected_goals","home_expected_goals"]:
                pass #no need to do
            elif stat['type'] == "Ball Possession":
                converted_fixture['away_ball_possession_%'] = int(stat['value'][:-1])
            elif stat['type'] == "Yellow Cards" or stat['type'] == "Red Cards" or stat['type'] == "Goalkeeper Saves":
                if stat['value'] != None:
                    converted_fixture[new_label] = stat['value']
                else:
                    converted_fixture[new_label] = 0
            elif stat['type'] == "Passes %":
                converted_fixture['away_passing_accuracy_%'] = int(stat['value'][:-1])
            else:
                if stat['value'] == None:
                    converted_fixture[new_label] = 0
                else:
                    converted_fixture[new_label] = stat['value']
        
        final_data.append(converted_fixture)

    #save the file
    json_string = json.dumps(final_data, indent=4) 

    with open(f'../dataset/epl_numerical/epl_numerical_{i}.json','w') as outfile:
        outfile.write(json_string)
        print(str(i)+'success')
    

2015success
2016success
2017success
2018success
2019success
2020success
2021success
2022success
2023success


To find cumulative data for the team before each match in the season season-to-date

In [ ]:
import json
for i in range(2015,2024):
    with open(f'../dataset/epl_numerical/epl_numerical_{i}.json') as f:
        data = json.load(f)

    to_json = []

    for match in data:
        match_timestamp = match['timestamp']
        
        home_team_id = match['home_team_id']
        away_team_id = match['away_team_id']

        cumulative_data = {
            "fixture_id": match['fixture_id'],
            "timestamp": match['timestamp'],
            "home_team": match['home_team'],
            'home_team_id' : home_team_id,
            "away_team": match['away_team'],
            'away_team_id' : away_team_id,
            'previous_match_count_home': 0,
            'previous_match_count_away': 0,

            "avg_home_fulltime_score": 0,
            "avg_away_fulltime_score": 0,
            "avg_home_shots_on_goal": 0,
            "avg_home_shots_off_goal": 0,
            "avg_home_total_shots": 0,
            "avg_home_blocked_shots": 0,
            "avg_home_shots_insidebox": 0,
            "avg_home_shots_outsidebox": 0,
            "avg_home_fouls": 0,
            "avg_home_corner_kicks": 0,
            "avg_home_offsides": 0,
            "avg_home_ball_possession_%": 0,
            "avg_home_yellow_cards": 0,
            "avg_home_red_cards": 0,
            "avg_home_goalkeeper_saves": 0,
            "avg_home_passing_accuracy_%": 0,
            "avg_away_shots_on_goal": 0,
            "avg_away_shots_off_goal": 0,
            "avg_away_total_shots": 0,
            "avg_away_blocked_shots": 0,
            "avg_away_shots_insidebox": 0,
            "avg_away_shots_outsidebox": 0,
            "avg_away_fouls": 0,
            "avg_away_corner_kicks": 0,
            "avg_away_offsides": 0,
            "avg_away_ball_possession_%": 0,
            "avg_away_yellow_cards": 0,
            "avg_away_red_cards": 0,
            "avg_away_goalkeeper_saves": 0,
            "avg_away_passing_accuracy_%": 0
        }

        

        for previous_match in data:
            previous_match_timestamp = previous_match['timestamp']
            previous_home_team_id = previous_match['home_team_id']
            previous_away_team_id = previous_match['away_team_id']
            home_counted = False
            away_counted = False

            #check home team first
            #home to home
            if match_timestamp > previous_match_timestamp and home_team_id==previous_home_team_id:
                home_counted = True

                for attribute in previous_match:
                    if 'home' in attribute: #select only home attributes from previous match
                        edited_attribute_name = 'avg_' + attribute

                        if edited_attribute_name in cumulative_data:
                            cumulative_data[edited_attribute_name] += previous_match[attribute]

            #away to home
            elif match_timestamp > previous_match_timestamp and home_team_id==previous_away_team_id:
                home_counted = True

                for attribute in previous_match:
                    if 'away' in attribute: #select only away attributes from previous match
                        edited_attribute_name = 'avg_' + attribute
                        edited_attribute_name = edited_attribute_name.replace('away','home')

                        if edited_attribute_name in cumulative_data:
                            cumulative_data[edited_attribute_name] += previous_match[attribute]

            #check away team next
            #home to away
            if match_timestamp > previous_match_timestamp and away_team_id==previous_home_team_id:
                away_counted = True

                for attribute in previous_match:
                    if 'home' in attribute: #select only home attributes from previous match
                        edited_attribute_name = 'avg_' + attribute
                        edited_attribute_name = edited_attribute_name.replace('home','away')

                        if edited_attribute_name in cumulative_data:
                            cumulative_data[edited_attribute_name] += previous_match[attribute]

            #away to away
            elif match_timestamp > previous_match_timestamp and away_team_id==previous_away_team_id:
                away_counted = True

                for attribute in previous_match:
                    if 'away' in attribute: #select only away attributes from previous match
                        edited_attribute_name = 'avg_' + attribute

                        if edited_attribute_name in cumulative_data:
                            cumulative_data[edited_attribute_name] += previous_match[attribute]

            if home_counted==True:
                cumulative_data['previous_match_count_home'] += 1
            if away_counted==True:
                cumulative_data['previous_match_count_away'] += 1
        
        #average all the data
        divide_by_home = cumulative_data['previous_match_count_home']
        divide_by_away = cumulative_data['previous_match_count_away']

        exclude = ["fixture_id",
                    "timestamp",
                    "home_team",
                    'home_team_id',
                    "away_team",
                    'away_team_id',
                    'previous_match_count_home',
                    'previous_match_count_away']
        
        for k in cumulative_data:
            if k not in exclude:
                if cumulative_data['previous_match_count_home'] != 0 and 'home' in k:
                    cumulative_data[k] = round(cumulative_data[k]/divide_by_home,2)
                elif cumulative_data['previous_match_count_away'] != 0 and 'away' in k:
                    cumulative_data[k] = round(cumulative_data[k]/divide_by_away,2)

        to_json.append(cumulative_data)
    
    #save the file
    json_string = json.dumps(to_json, indent=4) 

    with open(f'../dataset/epl_cumulative/epl_cumulative_{i}.json','w') as outfile:
        outfile.write(json_string)
        print(str(i)+'success')

2015success
2016success
2017success
2018success
2019success
2020success
2021success
2022success
2023success


To find last 5 match data for the team before each match in the season season-to-date. If there are less than 5 matches before the match, all fields will be populated with 0

In [ ]:
import json
for i in range(2015,2024):
    with open(f'../dataset/epl_numerical/epl_numerical_{i}.json') as f:
        data = json.load(f)

    to_json = []

    for match in data:
        match_timestamp = match['timestamp']
        
        home_team_id = match['home_team_id']
        away_team_id = match['away_team_id']

        cumulative_data = {
            "fixture_id": match['fixture_id'],
            "timestamp": match['timestamp'],
            "home_team": match['home_team'],
            'home_team_id' : home_team_id,
            "away_team": match['away_team'],
            'away_team_id' : away_team_id,
            'previous_match_count_home': 0,
            'previous_match_count_away': 0,

            "avg_home_fulltime_score": 0,
            "avg_away_fulltime_score": 0,
            "avg_home_shots_on_goal": 0,
            "avg_home_shots_off_goal": 0,
            "avg_home_total_shots": 0,
            "avg_home_blocked_shots": 0,
            "avg_home_shots_insidebox": 0,
            "avg_home_shots_outsidebox": 0,
            "avg_home_fouls": 0,
            "avg_home_corner_kicks": 0,
            "avg_home_offsides": 0,
            "avg_home_ball_possession_%": 0,
            "avg_home_yellow_cards": 0,
            "avg_home_red_cards": 0,
            "avg_home_goalkeeper_saves": 0,
            "avg_home_passing_accuracy_%": 0,
            "avg_away_shots_on_goal": 0,
            "avg_away_shots_off_goal": 0,
            "avg_away_total_shots": 0,
            "avg_away_blocked_shots": 0,
            "avg_away_shots_insidebox": 0,
            "avg_away_shots_outsidebox": 0,
            "avg_away_fouls": 0,
            "avg_away_corner_kicks": 0,
            "avg_away_offsides": 0,
            "avg_away_ball_possession_%": 0,
            "avg_away_yellow_cards": 0,
            "avg_away_red_cards": 0,
            "avg_away_goalkeeper_saves": 0,
            "avg_away_passing_accuracy_%": 0
        }

        
        eligible_previous_matches = []

        for previous_match in data:

            if match['timestamp'] > previous_match['timestamp']:

                eligible_previous_matches.append(previous_match)
        
        #sort eligible previous matches by recency, the most recent in front
        def get_timestamp(e):
            return e['timestamp']
        eligible_previous_matches.sort(key=get_timestamp, reverse=True)

        

        for previous_match in eligible_previous_matches:

            
            previous_match_timestamp = previous_match['timestamp']
            previous_home_team_id = previous_match['home_team_id']
            previous_away_team_id = previous_match['away_team_id']
            home_counted = False
            away_counted = False

            #check home team first
            #home to home
            if cumulative_data['previous_match_count_home'] < 5: #only take up to the five most recent matches

                if match_timestamp > previous_match_timestamp and home_team_id==previous_home_team_id:
                    home_counted = True

                    for attribute in previous_match:
                        if 'home' in attribute: #select only home attributes from previous match
                            edited_attribute_name = 'avg_' + attribute

                            if edited_attribute_name in cumulative_data:
                                cumulative_data[edited_attribute_name] += previous_match[attribute]

                #away to home
                elif match_timestamp > previous_match_timestamp and home_team_id==previous_away_team_id:
                    home_counted = True

                    for attribute in previous_match:
                        if 'away' in attribute: #select only away attributes from previous match
                            edited_attribute_name = 'avg_' + attribute
                            edited_attribute_name = edited_attribute_name.replace('away','home')

                            if edited_attribute_name in cumulative_data:
                                cumulative_data[edited_attribute_name] += previous_match[attribute]

            #check away team next
            #home to away
            if cumulative_data['previous_match_count_away'] < 5: #only take up to the five most recent matches
                
                if match_timestamp > previous_match_timestamp and away_team_id==previous_home_team_id:
                    away_counted = True

                    for attribute in previous_match:
                        if 'home' in attribute: #select only home attributes from previous match
                            edited_attribute_name = 'avg_' + attribute
                            edited_attribute_name = edited_attribute_name.replace('home','away')

                            if edited_attribute_name in cumulative_data:
                                cumulative_data[edited_attribute_name] += previous_match[attribute]

                #away to away
                elif match_timestamp > previous_match_timestamp and away_team_id==previous_away_team_id:
                    away_counted = True

                    for attribute in previous_match:
                        if 'away' in attribute: #select only away attributes from previous match
                            edited_attribute_name = 'avg_' + attribute

                            if edited_attribute_name in cumulative_data:
                                cumulative_data[edited_attribute_name] += previous_match[attribute]

            if home_counted==True:
                cumulative_data['previous_match_count_home'] += 1
            if away_counted==True:
                cumulative_data['previous_match_count_away'] += 1

        #average all the data
        divide_by_home = cumulative_data['previous_match_count_home']
        divide_by_away = cumulative_data['previous_match_count_away']

        exclude = ["fixture_id",
                    "timestamp",
                    "home_team",
                    'home_team_id',
                    "away_team",
                    'away_team_id',
                    'previous_match_count_home',
                    'previous_match_count_away']
        
        for k in cumulative_data:
            if k not in exclude:
                if cumulative_data['previous_match_count_home'] != 0 and 'home' in k:
                    cumulative_data[k] = round(cumulative_data[k]/divide_by_home,2)
                elif cumulative_data['previous_match_count_away'] != 0 and 'away' in k:
                    cumulative_data[k] = round(cumulative_data[k]/divide_by_away,2)

        if divide_by_home == 5 and divide_by_away == 5:
            to_json.append(cumulative_data)
    
    #save the file
    json_string = json.dumps(to_json, indent=4) 

    with open(f'../dataset/epl_cumulative/epl_cumulative_{i}.json','w') as outfile:
        outfile.write(json_string)
        print(str(i)+'success')

2015success
2016success
2017success
2018success
2019success
2020success
2021success
2022success
2023success


To get the difference between teams in the cumulative data

In [ ]:
import json
for i in range(2015,2024):
    data=None
    with open(f'../dataset/epl_cumulative_5/epl_cumulative_5_{i}.json') as f:
        data = json.load(f)
    
    exclude = [
        "fixture_id",
        "timestamp",
        "home_team",
        "home_team_id",
        "away_team",
        "away_team_id",
        "previous_match_count_home",
        "previous_match_count_away"
    ]

    to_json = []

    for fixture in data:

        difference_fixture = {
            "fixture_id" : fixture["fixture_id"],
            "timestamp" : fixture["timestamp"],
            "home_team" : fixture["home_team"],
            "home_team_id" : fixture["home_team_id"],
            "away_team" : fixture["away_team"],
            "away_team_id" : fixture["away_team_id"]
        }

        for attribute in fixture:

            if attribute in exclude:
                continue

            elif 'home' in attribute:
                
                away_attribute = attribute.replace('home','away')
                diff_attribute = attribute.replace('home','diff_5')

                #home avg minus away diff
                difference_fixture[diff_attribute] = round(fixture[attribute] - fixture[away_attribute],2)
        
        to_json.append(difference_fixture)
    
    #save the file
    json_string = json.dumps(to_json, indent=4) 

    with open(f'../dataset/epl_cumulative_diff_5/epl_cumulative_diff_5_{i}.json','w') as outfile:
        outfile.write(json_string)
        print(str(i)+'success')


2015success
2016success
2017success
2018success
2019success
2020success
2021success
2022success
2023success
